In [13]:
import importlib
import utils
importlib.reload(utils)
import pandas as pd
import tensorflow as tf
tf.test.is_built_with_cuda()#GPU aktif mi?

True

In [7]:
print("TensorFlow version:", tf.__version__)
print("GPU Build Info:", tf.sysconfig.get_build_info())
print("GPU Devices:", tf.config.list_physical_devices('GPU'))

# Bellek optimizasyonu (GPU kullanımı için)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Bellek büyümesini sınırla
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)#TensorFlow’un başlangıçta minimum bellek tahsis etmesini ve yalnızca gerektiğinde bellek kullanmasını sağlar.
    except RuntimeError as e:
        print(e)

TensorFlow version: 2.10.0
GPU Build Info: OrderedDict([('cpu_compiler', 'C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/HostX64/x64/cl.exe'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '64_112'), ('cudart_dll_name', 'cudart64_112.dll'), ('cudnn_dll_name', 'cudnn64_8.dll'), ('cudnn_version', '64_8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', False), ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll'), ('nvcuda_dll_name', 'nvcuda.dll')])
GPU Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
#Output konsol ayarlamaları
pd.set_option('display.max_columns', None)# Tüm sütunları göster
pd.set_option('display.max_rows', None)# Tüm satırları göster
pd.set_option('display.max_colwidth', None)# Tüm sütun içeriğini gösterir.
pd.set_option("display.float_format", lambda x: '%.3f' % x)# ondalıklı sayıları 3 basamaklı şekilde kısaltır.
pd.set_option('display.width', 5000)# DataFrame oto. alta geçmesin tek satırda gösterilsin diye. 5000 karakter hakkı vardır.

In [10]:
IMG_SIZE = (224, 224)   # Güncellenmiş boyut
BATCH_SIZE = 32 # Her iterasyonda işlenecek olan gruplandırılmış görüntü sayısıdır.
EPOCHS = 30 # Modelin veriyi kaç defa işleyecek
BASE_PATH = "../TUBITAK/datasets/CSV_Multi_Label_Classification"

In [26]:
#Veri Setleri Dağılımı ve Eğitim Seti İçeriği
train_df = utils.load_data('train')
valid_df = utils.load_data('valid')
test_df = utils.load_data('test')

print(f"Train set: {len(train_df)} image")
print(f"Validation set: {len(valid_df)} image")
print(f"Test set: {len(test_df)} image")

# Etiket dağılımını kontrol et
print("Data sütunları:", train_df.columns.tolist())
print("\nEğitim seti etiket dağılımı:")
print("Moderate Accident:", train_df['moderate'].sum())
print("Severe Accident:", train_df['severe'].sum())
print("No Accident:", train_df['no_accident'].sum())
print("Multi-label (Moderate & Severe):", ((train_df['moderate'] == 1) & (train_df['severe'] == 1)).sum())

# Tensorflow Dataset'leri oluştur
train_dataset = utils.create_dataset(train_df)
valid_dataset = utils.create_dataset(valid_df)
test_dataset = utils.create_dataset(test_df)

print("\nİlk birkaç satır:")
train_df.head()

Train set: 10469 image
Validation set: 1004 image
Test set: 649 image
Data sütunları: ['filename', 'moderate', 'severe', 'no_accident']

Eğitim seti etiket dağılımı:
Moderate Accident: 3402
Severe Accident: 6853
No Accident: 259
Multi-label (Moderate & Severe): 45

İlk birkaç satır:


,filename,moderate,severe,no_accident
0,../TUBITAK/datasets/CSV_Multi_Label_Classification\train\FLIP_CAR_V160_jpg.rf.38d0202f0df2947477b997dc3386e988.jpg,1,1,0
1,../TUBITAK/datasets/CSV_Multi_Label_Classification\train\images-18_jpg.rf.3891ad2ae29961a8d9e95f1900286b9e.jpg,0,1,0
2,../TUBITAK/datasets/CSV_Multi_Label_Classification\train\images-3_jpg.rf.390be461aa11e350bf6d77d460424853.jpg,0,1,0
3,../TUBITAK/datasets/CSV_Multi_Label_Classification\train\severityV3_24_jpeg_jpg.rf.388f61747f536c0e3eeba7090719cf72.jpg,0,1,0
4,../TUBITAK/datasets/CSV_Multi_Label_Classification\train\accidentFrame3008_jpg.rf.38dba20c29c95c8fa02b82586a341ff2.jpg,1,0,0
